Esse jupyter notebook serve para transformar os dados brutos zipados em arquivos agregados

In [1]:
import pandas as pd
import zipfile
import os

%cd ..

## Bolsa Família

In [16]:
# de 2020-01 até 2021-10

file_paths_bf = [    
    'raw_data/files/bolsa_familia/201908',
    'raw_data/files/bolsa_familia/201909',
    'raw_data/files/bolsa_familia/201910',
    'raw_data/files/bolsa_familia/201911',
    'raw_data/files/bolsa_familia/201912',
    'raw_data/files/bolsa_familia/202001',
    'raw_data/files/bolsa_familia/202002',
    'raw_data/files/bolsa_familia/202003',
    'raw_data/files/bolsa_familia/202004',
    'raw_data/files/bolsa_familia/202005',
    'raw_data/files/bolsa_familia/202006',
    'raw_data/files/bolsa_familia/202007',
    'raw_data/files/bolsa_familia/202008',
    'raw_data/files/bolsa_familia/202009',
    'raw_data/files/bolsa_familia/202010',
    'raw_data/files/bolsa_familia/202011',
    'raw_data/files/bolsa_familia/202012',
    'raw_data/files/bolsa_familia/202101',
    'raw_data/files/bolsa_familia/202102',
    'raw_data/files/bolsa_familia/202103',
    'raw_data/files/bolsa_familia/202104',
    'raw_data/files/bolsa_familia/202105',
    'raw_data/files/bolsa_familia/202106',
    'raw_data/files/bolsa_familia/202107',
    'raw_data/files/bolsa_familia/202108',
    'raw_data/files/bolsa_familia/202109',
    'raw_data/files/bolsa_familia/202110',
]

In [17]:
def extract_zip_file_bf(path):
    if os.path.exists(path):
        with zipfile.ZipFile(path, 'r') as zip_ref:
            zip_ref.extractall('raw_data/bolsa_familia/')
    else:
        print(f"File {path} doesn't exist.")

In [18]:
def create_agg_chunks_bf(path_mes):
    tamanho_total = 0
    with pd.read_csv('raw_data/bolsa_familia/' + path_mes + '_BolsaFamilia_Pagamentos.csv', chunksize=500_000, sep=';', encoding='latin-1') as reader:
            clean_chunks = []
            for df in reader:
                tamanho_total += len(df)
                df['VALOR PARCELA'] = df['VALOR PARCELA'].str.replace(',','.')
                df['VALOR PARCELA'] = df['VALOR PARCELA'].astype(float)

                colunas = ['CÓDIGO MUNICÍPIO SIAFI', 'VALOR PARCELA', 'NOME FAVORECIDO']

                valores_soma = df[colunas].groupby(['CÓDIGO MUNICÍPIO SIAFI'],dropna=False).sum(numeric_only=True).reset_index()
                valores_count = df[colunas].groupby(['CÓDIGO MUNICÍPIO SIAFI', "NOME FAVORECIDO"],dropna=False).size().groupby(['CÓDIGO MUNICÍPIO SIAFI']).size().reset_index()

                df_merge = valores_soma.merge(valores_count, on='CÓDIGO MUNICÍPIO SIAFI', how='outer')
                df_merge.columns = ['municipio_siafi', 'soma', 'contagem']
                clean_chunks.append(df_merge)
    return clean_chunks, tamanho_total

def merge_chunks_bf(clean_chunks):
    df = pd.DataFrame()

    for chunk in clean_chunks:
        if df.empty:
            df = chunk
        else:
            df_merge = df.merge(chunk, on='municipio_siafi', how='outer')
            assert len(df_merge) <= len(df) + len(chunk)
            df_merge = df_merge.fillna(0)
            df_merge['contagem'] = df_merge['contagem_x'] + df_merge['contagem_y']
            df_merge['soma'] = df_merge['soma_x'] + df_merge['soma_y']
            df_merge = df_merge.drop(['soma_x', 'soma_y', 'contagem_x', 'contagem_y'], axis=1)
            df = df_merge
    return df

def transform_file_bf(path):
    path_mes = path.split('/')[-1]

    # Check if transformed file already exists
    file_destination_name = 'parsed_data/bolsa_familia/' + path_mes + '.csv'

    if os.path.exists(file_destination_name):
        print(f'Arquivo {file_destination_name} já existente, pulando transformação.')
        return
    
    # Check if extracted file already exists
    extracted_file = 'raw_data/bolsa_familia/' + path_mes + '_BolsaFamilia_Pagamentos.csv'
    if not os.path.exists(extracted_file):
        print(f'Extraindo {path}.')
        extract_zip_file_bf(path)
        print(f'Concluído.')
    else:  
        print(f'Dados já extraídos, reaproveitando {extracted_file}.')
    
    print(f'Agregando dados.')

    clean_chunks, tamanho_total = create_agg_chunks_bf(path_mes)

    print(f'tamanho total: {tamanho_total}')

    df_final = merge_chunks_bf(clean_chunks)

    print(f'Concluído.')
    df_final.to_csv(file_destination_name, index=False)
    print(f'Arquivo salvo em {file_destination_name}.')

    # Clean extracted file in the end
    print(f'Apagando csv extraído.')
    os.remove(extracted_file)
    print(f'Concluído.')


In [19]:
for file in file_paths_bf:
    transform_file_bf(file)

Arquivo parsed_data/bolsa_familia/201908.csv já existente, pulando transformação.
Arquivo parsed_data/bolsa_familia/201909.csv já existente, pulando transformação.
Arquivo parsed_data/bolsa_familia/201910.csv já existente, pulando transformação.
Arquivo parsed_data/bolsa_familia/201911.csv já existente, pulando transformação.
Arquivo parsed_data/bolsa_familia/201912.csv já existente, pulando transformação.
Arquivo parsed_data/bolsa_familia/202001.csv já existente, pulando transformação.
Arquivo parsed_data/bolsa_familia/202002.csv já existente, pulando transformação.
Arquivo parsed_data/bolsa_familia/202003.csv já existente, pulando transformação.
Arquivo parsed_data/bolsa_familia/202004.csv já existente, pulando transformação.
Arquivo parsed_data/bolsa_familia/202005.csv já existente, pulando transformação.
Arquivo parsed_data/bolsa_familia/202006.csv já existente, pulando transformação.
Arquivo parsed_data/bolsa_familia/202007.csv já existente, pulando transformação.
Arquivo parsed_d

## Auxílio Emergencial

In [2]:
# de 2020-01 até 2021-10

file_paths_ae = [    
    'raw_data/files/auxilio_emergencial/202206',
    'raw_data/files/auxilio_emergencial/202205',
    'raw_data/files/auxilio_emergencial/202204',
    'raw_data/files/auxilio_emergencial/202203',
    'raw_data/files/auxilio_emergencial/202202',
    'raw_data/files/auxilio_emergencial/202201',
    'raw_data/files/auxilio_emergencial/202112',
    'raw_data/files/auxilio_emergencial/202111',
    'raw_data/files/auxilio_emergencial/202110',
    'raw_data/files/auxilio_emergencial/202109',
    'raw_data/files/auxilio_emergencial/202108',
    'raw_data/files/auxilio_emergencial/202107',
    'raw_data/files/auxilio_emergencial/202106',
    'raw_data/files/auxilio_emergencial/202105',
    'raw_data/files/auxilio_emergencial/202104',
    'raw_data/files/auxilio_emergencial/202103',
    'raw_data/files/auxilio_emergencial/202102',
    'raw_data/files/auxilio_emergencial/202101',
    'raw_data/files/auxilio_emergencial/202012',
    'raw_data/files/auxilio_emergencial/202011',
    'raw_data/files/auxilio_emergencial/202010',
    'raw_data/files/auxilio_emergencial/202009',
    'raw_data/files/auxilio_emergencial/202008',
    'raw_data/files/auxilio_emergencial/202007',
    'raw_data/files/auxilio_emergencial/202006',
    'raw_data/files/auxilio_emergencial/202005',
    'raw_data/files/auxilio_emergencial/202004',
]

In [3]:
def extract_zip_file_ae(path):
    if os.path.exists(path):
        with zipfile.ZipFile(path, 'r') as zip_ref:
            zip_ref.extractall('raw_data/auxilio_emergencial/')
    else:
        print(f"File {path} doesn't exist.")

def create_agg_chunks_ae(path_mes):
    tamanho_total = 0
    with pd.read_csv('raw_data/auxilio_emergencial/' + path_mes + '_AuxilioEmergencial.csv', chunksize=500_000, sep=';', encoding='latin-1') as reader:
            clean_chunks = []
            for df in reader:
                df['VALOR BENEFÍCIO'] = df['VALOR BENEFÍCIO'].str.replace(',','.')
                df['VALOR BENEFÍCIO'] = df['VALOR BENEFÍCIO'].astype(float)

                obs_filter = (df['OBSERVAÇÃO'] == 'Não há') | (df['OBSERVAÇÃO'].isna())

                df = df[obs_filter]
                
                tamanho_total += len(df)

                colunas = ['CÓDIGO MUNICÍPIO IBGE', 'VALOR BENEFÍCIO', 'NOME BENEFICIÁRIO']

                valores_soma = df[colunas].groupby(['CÓDIGO MUNICÍPIO IBGE'],dropna=False).sum().reset_index()
                valores_count = df[colunas].groupby(['CÓDIGO MUNICÍPIO IBGE', 'NOME BENEFICIÁRIO'], dropna=False).size().groupby(['CÓDIGO MUNICÍPIO IBGE']).size().reset_index()

                df_merge = valores_soma.merge(valores_count, on='CÓDIGO MUNICÍPIO IBGE', how='outer')
                df_merge.columns = ['municipio_ibge', 'soma', 'contagem']
                clean_chunks.append(df_merge)
    return clean_chunks, tamanho_total

def merge_chunks_ae(clean_chunks):
    df = pd.DataFrame()

    for chunk in clean_chunks:
        if df.empty:
            df = chunk
        else:
            df_merge = df.merge(chunk, on='municipio_ibge', how='outer')
            assert len(df_merge) <= len(df) + len(chunk)
            df_merge = df_merge.fillna(0)
            df_merge['contagem'] = df_merge['contagem_x'] + df_merge['contagem_y']
            df_merge['soma'] = df_merge['soma_x'] + df_merge['soma_y']
            df_merge = df_merge.drop(['soma_x', 'soma_y', 'contagem_x', 'contagem_y'], axis=1)
            df = df_merge
    return df

def transform_file_ae(path):
    path_mes = path.split('/')[-1]

    # Check if transformed file already exists
    file_destination_name = 'parsed_data/auxilio_emergencial/' + path_mes + '.csv'

    if os.path.exists(file_destination_name):
        print(f'Arquivo {file_destination_name} já existente, pulando.. transformação.')
        return
    
    # Check if extracted file already exists
    extracted_file = 'raw_data/auxilio_emergencial/' + path_mes + '_AuxilioEmergencial.csv'
    if not os.path.exists(extracted_file):
        print(f'Extraindo {path}.')
        extract_zip_file_ae(path)
        print(f'Concluído.')
    else:  
        print(f'Dados já extraídos, reaproveitando {extracted_file}.')
    
    print(f'Agregando dados.')

    clean_chunks, tamanho_total = create_agg_chunks_ae(path_mes)

    print(f'tamanho total: {tamanho_total}')

    df_final = merge_chunks_ae(clean_chunks)

    print(f'Concluído.')
    df_final.to_csv(file_destination_name, index=False)
    print(f'Arquivo salvo em {file_destination_name}.')

    # Clean extracted file in the end
    print(f'Apagando csv extraído.')
    os.remove(extracted_file)
    print(f'Concluído.')

In [5]:
for file in file_paths_ae:
    transform_file_ae(file)

Arquivo parsed_data/auxilio_emergencial/202206.csv já existente, pulando.. transformação.
Arquivo parsed_data/auxilio_emergencial/202205.csv já existente, pulando.. transformação.
Arquivo parsed_data/auxilio_emergencial/202204.csv já existente, pulando.. transformação.
Arquivo parsed_data/auxilio_emergencial/202203.csv já existente, pulando.. transformação.
Arquivo parsed_data/auxilio_emergencial/202202.csv já existente, pulando.. transformação.
Arquivo parsed_data/auxilio_emergencial/202201.csv já existente, pulando.. transformação.
Arquivo parsed_data/auxilio_emergencial/202112.csv já existente, pulando.. transformação.
Arquivo parsed_data/auxilio_emergencial/202111.csv já existente, pulando.. transformação.
Arquivo parsed_data/auxilio_emergencial/202110.csv já existente, pulando.. transformação.
Arquivo parsed_data/auxilio_emergencial/202109.csv já existente, pulando.. transformação.
Arquivo parsed_data/auxilio_emergencial/202108.csv já existente, pulando.. transformação.
Arquivo pa